In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import datetime
import requests
from datetime import datetime,timedelta
import time

In [3]:
resale_2012_2014 = pd.read_csv('resale-flat-prices/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv',
                               parse_dates=True)
resale_2015_2016 = pd.read_csv('resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv',
                               parse_dates=True)
resale_2017 = pd.read_csv('resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv',
                               parse_dates=True)

In [4]:
df = pd.concat([resale_2012_2014,resale_2015_2016,resale_2017],ignore_index = True)
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,NaN
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,NaN
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,NaN
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,NaN
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
224458,2022-09,YISHUN,5 ROOM,513B,YISHUN ST 51,13 TO 15,113.0,Improved,2018,716000.0,94 years 06 months
224459,2022-09,YISHUN,5 ROOM,505B,YISHUN ST 51,07 TO 09,113.0,Improved,2016,636000.0,92 years 07 months
224460,2022-09,YISHUN,5 ROOM,716,YISHUN ST 71,04 TO 06,131.0,Improved,1987,600000.0,64 years 02 months
224461,2022-09,YISHUN,5 ROOM,713,YISHUN ST 71,04 TO 06,122.0,Improved,1987,532000.0,63 years 10 months


In [5]:
df_storey =df['storey_range'].str.split(pat = " TO ", expand=True)

In [6]:
df_storey = df_storey.astype(float)
df_storeyrange =(df_storey[1] + df_storey[0])/2
df_storeyrange.head()

0    8.0
1    3.0
2    8.0
3    3.0
4    8.0
dtype: float64

In [7]:
features = df.drop(columns = ['storey_range','remaining_lease'], axis = 1)
features = pd.concat([features,df_storeyrange],axis=1)
features = features.rename(columns={0: "Storey"})
features.head()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,Storey
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,45.0,Improved,1986,250000.0,8.0
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,44.0,Improved,1980,265000.0,3.0
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,68.0,New Generation,1980,315000.0,8.0
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,67.0,New Generation,1984,320000.0,3.0
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,67.0,New Generation,1980,321000.0,8.0


In [8]:
features['address'] = features['block'] + " " + features['street_name']
addresslist = list(features['address'])
features.head()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,Storey,address
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,45.0,Improved,1986,250000.0,8.0,172 ANG MO KIO AVE 4
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,44.0,Improved,1980,265000.0,3.0,510 ANG MO KIO AVE 8
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,68.0,New Generation,1980,315000.0,8.0,610 ANG MO KIO AVE 4
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,67.0,New Generation,1984,320000.0,3.0,474 ANG MO KIO AVE 10
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,67.0,New Generation,1980,321000.0,8.0,604 ANG MO KIO AVE 5


In [9]:
features['lease_commence_dt']=pd.to_datetime(features['lease_commence_date'],format="%Y")
features['month'] = pd.to_datetime(features['month'],format="%Y-%m")
features.head()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,Storey,address,lease_commence_dt
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,45.0,Improved,1986,250000.0,8.0,172 ANG MO KIO AVE 4,1986-01-01
1,2012-03-01,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,44.0,Improved,1980,265000.0,3.0,510 ANG MO KIO AVE 8,1980-01-01
2,2012-03-01,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,68.0,New Generation,1980,315000.0,8.0,610 ANG MO KIO AVE 4,1980-01-01
3,2012-03-01,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,67.0,New Generation,1984,320000.0,3.0,474 ANG MO KIO AVE 10,1984-01-01
4,2012-03-01,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,67.0,New Generation,1980,321000.0,8.0,604 ANG MO KIO AVE 5,1980-01-01


In [10]:
features['current_lease_dt']=features['month']-features['lease_commence_dt']
features['current_lease_year'] = features['current_lease_dt'] / np.timedelta64(1, 'Y')
features['remaining_lease_year'] = 99-features['current_lease_year']
features.head()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,resale_price,Storey,address,lease_commence_dt,current_lease_dt,current_lease_year,remaining_lease_year
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,45.0,Improved,1986,250000.0,8.0,172 ANG MO KIO AVE 4,1986-01-01,9556 days,26.163439,72.836561
1,2012-03-01,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,44.0,Improved,1980,265000.0,3.0,510 ANG MO KIO AVE 8,1980-01-01,11748 days,32.164932,66.835068
2,2012-03-01,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,68.0,New Generation,1980,315000.0,8.0,610 ANG MO KIO AVE 4,1980-01-01,11748 days,32.164932,66.835068
3,2012-03-01,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,67.0,New Generation,1984,320000.0,3.0,474 ANG MO KIO AVE 10,1984-01-01,10287 days,28.164849,70.835151
4,2012-03-01,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,67.0,New Generation,1980,321000.0,8.0,604 ANG MO KIO AVE 5,1980-01-01,11748 days,32.164932,66.835068


In [11]:
features = features.drop(columns = ['current_lease_year','current_lease_dt','lease_commence_date']
                         ,axis=1)
features.sample()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,resale_price,Storey,address,lease_commence_dt,remaining_lease_year
53380,2015-01-01,WOODLANDS,EXECUTIVE,328,WOODLANDS ST 32,163.0,Apartment,628000.0,2.0,328 WOODLANDS ST 32,1994-01-01,78.000253


In [12]:
def getcoordinates(address):
    req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    resultsdict = eval(req.text)
    if len(resultsdict['results'])>0:
        return resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE']
    else:
        pass

In [22]:
old_df = pd.read_csv("model/datawithlatlong.csv",index_col='Unnamed: 0')

In [23]:
new_df = features[~features.index.isin(old_df.index)]

In [26]:
new_df = new_df.assign(Coords = np.NaN)

In [41]:
getcoordinates("100 AM")

('1.27468281482263', '103.843488359469')

In [ ]:
# On google the length of 1 degree is about 111km



In [30]:
new_df.iloc[0:10,-1] = new_df.iloc[0:10]['address'].apply(getcoordinates)

In [20]:
%%time
# plus 1 to get last part as the quotient*250 will be smaller than the length of the dataframe
# 250 is the rate limit per minute

x= (len(new_df) // 250) + 1

for i in range(x):
    # slice dataframe into blocks of 250
    # set min and max to prevent out of bounds error
    a = max(i*250,0)
    b = min((i+1)*250,len(new_df))

    # control limit to 250 per minute

    start_time = datetime.now()
    plus_1_min = start_time + timedelta(minutes = 1)
    new_df.iloc[0:250]['coords'] = new_df.iloc[0:250]['address'].apply(getcoordinates)
    difference = plus_1_min - datetime.now()

    # if loop takes longer than 1 min, do not wait

    if difference.days < 0:
        continue
    else:
        time.sleep(difference.seconds/60)


<timed exec>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

CPU times: total: 4min 26s
Wall time: 35min 46s


In [1]:
new_df

NameError: name 'new_df' is not defined

In [47]:
%%time

getcoordinates("345 KANG CHING RD")

CPU times: total: 15.6 ms
Wall time: 65.9 ms


('1.34020748183852', '103.721534317406')

In [48]:
old_df

,month,town,flat_type,floor_area_sqm,flat_model,resale_price,Storey,lease_commence_dt,remaining_lease_year,Latitude,Longitude
0,2012-03-01,ANG MO KIO,2 ROOM,45.0,Improved,250000.0,8.0,1986-01-01,72.836561,1.374694,103.836463
1,2012-03-01,ANG MO KIO,2 ROOM,44.0,Improved,265000.0,3.0,1980-01-01,66.835068,1.373401,103.849073
2,2012-03-01,ANG MO KIO,3 ROOM,68.0,New Generation,315000.0,8.0,1980-01-01,66.835068,1.379395,103.839157
3,2012-03-01,ANG MO KIO,3 ROOM,67.0,New Generation,320000.0,3.0,1984-01-01,70.835151,1.362758,103.858015
4,2012-03-01,ANG MO KIO,3 ROOM,67.0,New Generation,321000.0,8.0,1980-01-01,66.835068,1.379867,103.835977
...,...,...,...,...,...,...,...,...,...,...,...
197399,2021-09-01,YISHUN,5 ROOM,113.0,Improved,655000.0,11.0,2017-01-01,94.334606,1.415600,103.842414
197400,2021-09-01,YISHUN,5 ROOM,127.0,Improved,620000.0,5.0,1988-01-01,65.331958,1.416013,103.834369
197401,2021-09-01,YISHUN,5 ROOM,122.0,Improved,545000.0,5.0,1988-01-01,65.331958,1.414760,103.834649
197402,2021-09-01,YISHUN,EXECUTIVE,142.0,Apartment,648000.0,11.0,1987-01-01,64.332621,1.419829,103.836032


In [ ]:
coordinateslist= []
failed_address = []
count = 0
failed_count = 0
for address in addresslist:
    try:
        if len(getcoordinates(address))>0:
            count = count + 1
            #print('Extracting',count,'out of',len(addresslist),'addresses')
            coordinateslist.append(getcoordinates(address))
    except Exception as e:
        print(e)
        count = count + 1           
        failed_count = failed_count + 1
        print(address)
        print('Failed to extract',count,'out of',len(addresslist),'addresses')
        coordinateslist.append(add)
print('Total Number of Addresses With No Coordinates',failed_count)

In [41]:
df_coords = pd.DataFrame(coordinateslist)
df_combined = features.join(df_coords)
df_combined = df_combined.rename(columns={0: 'Latitude',1:'Longitude'})
df_combined

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,resale_price,Storey,address,lease_commence_dt,remaining_lease_year,Latitude,Longitude
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,45.0,Improved,250000.0,8.0,172 ANG MO KIO AVE 4,1986-01-01,72.836561,1.37469423247424,103.836462867365
1,2012-03-01,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,44.0,Improved,265000.0,3.0,510 ANG MO KIO AVE 8,1980-01-01,66.835068,1.37340092645025,103.849073244454
2,2012-03-01,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,68.0,New Generation,315000.0,8.0,610 ANG MO KIO AVE 4,1980-01-01,66.835068,1.37939470300077,103.839157232686
3,2012-03-01,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,67.0,New Generation,320000.0,3.0,474 ANG MO KIO AVE 10,1984-01-01,70.835151,1.36275784702216,103.858015323667
4,2012-03-01,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,67.0,New Generation,321000.0,8.0,604 ANG MO KIO AVE 5,1980-01-01,66.835068,1.37986658053966,103.835976650638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197399,2021-09-01,YISHUN,5 ROOM,511B,YISHUN ST 51,113.0,Improved,655000.0,11.0,511B YISHUN ST 51,2017-01-01,94.334606,1.41560014502104,103.84241412452
197400,2021-09-01,YISHUN,5 ROOM,841,YISHUN ST 81,127.0,Improved,620000.0,5.0,841 YISHUN ST 81,1988-01-01,65.331958,1.41601322618783,103.834368645699
197401,2021-09-01,YISHUN,5 ROOM,832,YISHUN ST 81,122.0,Improved,545000.0,5.0,832 YISHUN ST 81,1988-01-01,65.331958,1.41475962294169,103.834648982061
197402,2021-09-01,YISHUN,EXECUTIVE,614,YISHUN ST 61,142.0,Apartment,648000.0,11.0,614 YISHUN ST 61,1987-01-01,64.332621,1.4198290672055,103.836032497982


In [48]:
df_combined.loc[1207]

month                   2012-03-01 00:00:00
town                        KALLANG/WHAMPOA
flat_type                            3 ROOM
block                                     2
street_name                 ST. GEORGE'S RD
floor_area_sqm                           59
flat_model                         Improved
resale_price                         300000
Storey                                    3
address                   2 ST. GEORGE'S RD
lease_commence_dt       1976-01-01 00:00:00
remaining_lease_year                 62.835
Latitude                               None
Longitude                              None
Name: 1207, dtype: object

In [59]:
df_new = df_combined.dropna(axis=0,subset=['Latitude','Longitude'])
df_new = df_new.drop(columns=['block','street_name','address'])
df_new

,month,town,flat_type,floor_area_sqm,flat_model,resale_price,Storey,lease_commence_dt,remaining_lease_year,Latitude,Longitude
0,2012-03-01,ANG MO KIO,2 ROOM,45.0,Improved,250000.0,8.0,1986-01-01,72.836561,1.37469423247424,103.836462867365
1,2012-03-01,ANG MO KIO,2 ROOM,44.0,Improved,265000.0,3.0,1980-01-01,66.835068,1.37340092645025,103.849073244454
2,2012-03-01,ANG MO KIO,3 ROOM,68.0,New Generation,315000.0,8.0,1980-01-01,66.835068,1.37939470300077,103.839157232686
3,2012-03-01,ANG MO KIO,3 ROOM,67.0,New Generation,320000.0,3.0,1984-01-01,70.835151,1.36275784702216,103.858015323667
4,2012-03-01,ANG MO KIO,3 ROOM,67.0,New Generation,321000.0,8.0,1980-01-01,66.835068,1.37986658053966,103.835976650638
...,...,...,...,...,...,...,...,...,...,...,...
197399,2021-09-01,YISHUN,5 ROOM,113.0,Improved,655000.0,11.0,2017-01-01,94.334606,1.41560014502104,103.84241412452
197400,2021-09-01,YISHUN,5 ROOM,127.0,Improved,620000.0,5.0,1988-01-01,65.331958,1.41601322618783,103.834368645699
197401,2021-09-01,YISHUN,5 ROOM,122.0,Improved,545000.0,5.0,1988-01-01,65.331958,1.41475962294169,103.834648982061
197402,2021-09-01,YISHUN,EXECUTIVE,142.0,Apartment,648000.0,11.0,1987-01-01,64.332621,1.4198290672055,103.836032497982


In [60]:
df_new.to_csv('datawithlatlong.csv')